Detección de caras con webcam

In [2]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors

En el bucle de procesamiento, las teclas 'd' y 'e' `permiten respectivamenet cambiar de modelo de detección de caras, y en su caso de máscara de detección del rostro.

La ejecución de la siguiente celda produce error al no disponer de los archivos shape_predictor_5_face_landmarks.dat y shape_predictor_68_face_landmarks.dat que por su tamaño no se incluyeron en el repositorio. Pueden descargarse desde el enlace proporcionado en el campus virtual (opción aconsejada), o
desde el [repositorio de archivos de dlib](http://dlib.net/files/).



In [28]:

normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

    # Face detection and eye model setup
imodoF = 0
imodoE = 0

debug = 0

#Set camera resolution
cap.set(3,640);
cap.set(4,480);

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                # Show detected facial elements
                if imodoF > 0:
                    for (x, y) in shape:
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                        cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)


                # Normalize and show
                # color channels
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))
                cv2.imshow("Normalized", NormBGR)


    if debug:
        print("Processing time : {:.3f}".format(time.time() - t))

    # Show resulting image
    cv2.putText(frame, FDet.FaceDetectors[imodoF], (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    if imodoF == 1 or imodoF == 2:
        cv2.putText(frame, FDet.EyeDetectors[imodoE], (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break
    # Face detector change
    elif tec & 0xFF == ord('d'):
        imodoF = imodoF + 1
        if imodoF >= len(FDet.FaceDetectors):
            imodoF = 0
    #Eye detector change
    elif tec & 0xFF == ord('e'):
        imodoE = imodoE + 1
        if imodoE >= len(FDet.EyeDetectors):
            imodoE = 0

# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()

Camera 0


In [18]:
import dlib
from imutils import face_utils
import numpy as np
import cv2
import FaceNormalizationUtils as faceutils

def AddDecorate(frame,resizedBat,x,y):
    
    resizedBatMask = resizedBat[:,:,0]
    cv2.imshow("2",resized_image)
    regionFrame = frame[y:y+resizedBat.shape[0],x:x+resizedBat.shape[1]]
    BatMaskInv = cv2.bitwise_not(resizedBatMask)
    BgregionFrame = cv2.bitwise_and(regionFrame,regionFrame,mask=BatMaskInv)
    Bat = cv2.add(BgregionFrame,resizedBat)
    frame[y:y+resizedBat.shape[0],x:x+resizedBat.shape[1]] = Bat

def addPNG(frame,resized_image):
    
    n_frame = frame[y: y + h, x: x + w]
    #Establecemos la mascara para el video
    mask = resized_image[:,:,3]
    cv2.imshow("1",mask)

    # Invertir la máscara
    mask_inv = cv2.bitwise_not(mask)
    bg_black = cv2.bitwise_and(resized_image, resized_image, mask=mask)
    bg_black = bg_black[:, :, 0:3]
    bg_frame = cv2.bitwise_and(n_frame, n_frame, mask=mask_inv)

    #Resultado final y lo agregamos el frame
    final_pumpki = cv2.add(bg_black,bg_frame) 
    
    return final_pumpki
# My face detectors interface
def oscurecer_frame(frame):
    """
    Oscurece un frame sin que pierda el RGB.

    Args:
        frame: El frame a oscurecer.

    Returns:
        El frame oscurecido.
    """
    

    # Convertir el frame a escala de grises.
    img_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Aplicar un filtro de suavizado.
    img_suavizado = cv2.GaussianBlur(img_gris, (5, 5), 0)

    # Convertir el frame de escala de grises a RGB.
    img_rgb = cv2.cvtColor(img_suavizado, cv2.COLOR_GRAY2BGR)

    # Multiplicar cada canal RGB por un factor.
    img_rgb[:, :, 0] = img_rgb[:, :, 0] * 0.3
    img_rgb[:, :, 1] = img_rgb[:, :, 1] * 0.3
    img_rgb[:, :, 2] = img_rgb[:, :, 2] * 0.3

    return img_rgb
    
def rotate_image(image, angle):
    center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return rotated_image

FDet = FaceDetectors.FaceDetector()
cap = cv2.VideoCapture(0)

# Carga las imágenes de máscaras
mask_images = []
mask_images.append(cv2.imread("spookyPumpki2.png",cv2.IMREAD_UNCHANGED))
mask_images.append(cv2.imread("esqueleto.png",cv2.IMREAD_UNCHANGED))
mask_images.append(cv2.imread("scream.png",cv2.IMREAD_UNCHANGED))

current_mask_index = 0  # Índice de la máscara actual

#print('pumpkin.shape =',pumpkin.shape)
#cv2.imshow('pumpkin',pumpkin[:,:,3])
faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

while True:
  ret, frame = cap.read()
  frame = oscurecer_frame(frame)

  if ret == False:
    break
  
  faces = faceClassif.detectMultiScale(frame,1.3,5)

  values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[0], FDet.EyeDetectors[1])

    # Detectamos las caras en el fotograma.
  if values is not None:
    face, eyes, shape = values

  [x, y , w, h] = face

  if x > -1:
    #Redimensionar la imagen 

    resized_image = cv2.resize(mask_images[current_mask_index], (w, h), interpolation=cv2.INTER_AREA)
    
    #Aplicamos filtros para oscurecer la imagen
    
    imgOnFrame = addPNG(frame,resized_image)

  # Encuentra los landmarks faciales.
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)

    # Encuentra los puntos clave de los ojos.
    left_eye = shape[36:42]
    right_eye = shape[42:48]

   # Calcula el ángulo de inclinación de la cabeza.
    left_eye_center = left_eye.mean(axis=0).astype("int")
    right_eye_center = right_eye.mean(axis=0).astype("int")
    dY = left_eye_center[1] - right_eye_center[1]
    dX = left_eye_center[0] - right_eye_center[0]

    # Calcula el ángulo de inclinación de la cabeza en función de la posición de los ojos.
    if right_eye[0][0] > left_eye[0][0]:
        # La cabeza está inclinada hacia la izquierda, rota en sentido contrario.
        angle = -np.degrees(np.arctan2(dY, dX))
    else:
        # La cabeza está inclinada hacia la derecha, rota en sentido normal.
        angle = np.degrees(np.arctan2(dY, dX))


    # Rota la máscara según el ángulo de la cabeza utilizando tu función
    rotated_mask = rotate_image(imgOnFrame, angle)

    # Aplica una rotación adicional de 180 grados a la máscara para corregir la orientación
    rotated_mask = rotate_image(rotated_mask, 180)


    # Aplica la máscara rotada al frame
    frame[y: y + h, x: x + w] = rotated_mask

  resizedBat= cv2.resize(cv2.imread("bat.png"), (364//3,364//3), interpolation=cv2.INTER_AREA)

  AddDecorate(frame,resizedBat,10,10)
  AddDecorate(frame,resizedBat,500,30)


  cv2.imshow('Halloween',frame)

  key = cv2.waitKey(1) & 0xFF

  if key == 27:
    break
  elif key == 32:  # Barra espaciadora
        # Cambia a la siguiente máscara
        current_mask_index = (current_mask_index + 1) % len(mask_images)


# Liberamos los recursos
cap.release()
cv2.destroyAllWindows()

Camera 0
1/1 [==============================] - ETA: 0s

KeyboardInterrupt: 